In [1]:
from gensim.models import Word2Vec

In [2]:
!pip install emoji
!pip install contractions #for expanding texts like i'm to i am

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 14.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import nltk
import emoji
import contractions
nltk.download('stopwords') #for stopword
nltk.download('wordnet')
nltk.download('punkt') #pos tagging
nltk.download('averaged_perceptron_tagger')

from nltk.stem import WordNetLemmatizer
tweets = pd.DataFrame()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
tweets_data = pd.read_csv("/content/tweets.csv")
tweets_data

,Text
0,Just got my hands on the latest collection fro...
1,Disappointed with the quality of the fabric in...
2,Can't wait to strut my stuff in my new X Cloth...
3,The customer service at X Clothing Company is ...
4,Obsessed with the vibrant colors in the spring...
...,...
95,Just scored a killer deal on some workout gear...
96,Disappointed that X Clothing Company still has...
97,Obsessed with the boho-chic vibes of the new X...
98,The X Clothing Company app keeps crashing ever...


# Substitute emoji with their meaning

In [ ]:
tweet1 = tweets_data.iloc[0,0]

In [ ]:
tweets_data["Text"] = tweets_data["Text"].apply(emoji.demojize)

In [ ]:
tweets_data["Text"][0]

"Just got my hands on the latest collection from X Clothing Company, and I'm loving the sleek designs! :smiling_face_with_heart-eyes: #FashionForward\n"

# Removing the \n added by demojize

In [ ]:
tweets_data["Text"] = tweets_data["Text"].str.strip("\n")

In [ ]:
tweets_data["Text"][0]

"Just got my hands on the latest collection from X Clothing Company, and I'm loving the sleek designs! :smiling_face_with_heart-eyes: #FashionForward"

# Remove the punctuations

In [ ]:
import re

In [ ]:
def remove_punctuations(string):
  punc_removed = re.sub(r"[^\w\s']","",string)
  return punc_removed

In [ ]:
tweets_data["Text"] = tweets_data["Text"].apply(remove_punctuations)
tweets_data["Text"]

0     Just got my hands on the latest collection fro...
1     Disappointed with the quality of the fabric in...
2     Can't wait to strut my stuff in my new X Cloth...
3     The customer service at X Clothing Company is ...
4     Obsessed with the vibrant colors in the spring...
                            ...                        
95    Just scored a killer deal on some workout gear...
96    Disappointed that X Clothing Company still has...
97    Obsessed with the bohochic vibes of the new X ...
98    The X Clothing Company app keeps crashing ever...
99    Just discovered that X Clothing Company uses s...
Name: Text, Length: 100, dtype: object

# Removed the extra spaces

In [ ]:
tweets_data["Text"] = tweets_data["Text"].str.replace("  "," ")
tweets_data["Text"][0]

"Just got my hands on the latest collection from X Clothing Company and I'm loving the sleek designs smiling_face_with_hearteyes FashionForward"

#Tokenization

In [ ]:
contractions.contractions_dict

In [ ]:
contractions.fix("I'm")

'I am'

In [ ]:
# tweets_data["Text"] = tweets_data["Text"].apply(lambda tweet:nltk.word_tokenize(tweet))
# tweets_data["Text"][0]

In [ ]:
tweets_data["Text"] = tweets_data["Text"].apply(lambda tweet:tweet.split())
tweets_data["Text"][0]

['Just',
 'got',
 'my',
 'hands',
 'on',
 'the',
 'latest',
 'collection',
 'from',
 'X',
 'Clothing',
 'Company',
 'and',
 "I'm",
 'loving',
 'the',
 'sleek',
 'designs',
 'smiling_face_with_hearteyes',
 'FashionForward']

# Changing slangs and contractions to normal english

In [ ]:
apostrophes = contractions.contractions_dict

In [ ]:
contractions.fix("idk")

'i do not know'

In [ ]:
def remove_contractions(tokens):
  temp = tokens.copy()
  for index,token in enumerate(temp):
    if token in apostrophes.keys():

      temp.remove(token)
      expanded = contractions.fix(token)
      end_point = len(expanded) - 1
      temp[index:index+end_point] = expanded.split()

  return temp

In [ ]:
tweets_data["Text"] = tweets_data["Text"].apply(remove_contractions)
tweets_data["Text"][0]

['got',
 'hands',
 'latest',
 'collection',
 'X',
 'Clothing',
 'Company',
 'designs',
 'smiling_face_with_hearteyes',
 'FashionForward']

In [ ]:
temp_tokens = tweets_data["Text"]
model2 = Word2Vec(temp_tokens, min_count=1, vector_size=1,
                                window=5, sg=1)

In [ ]:
model2.wv.get_vector("smiling_face_with_hearteyes")

array([-0.62881535], dtype=float32)

# Stopword removal

In [ ]:
stopwords = nltk.corpus.stopwords.words("english")

In [ ]:
def remove_stopwords(tokens,stopwords):
  non_stopwords = [token for token in tokens if token.lower() not in stopwords]
  return non_stopwords

In [ ]:
tweets_data["Text"] = tweets_data["Text"].apply(remove_stopwords,args=([stopwords]))
tweets_data["Text"][0]

['got',
 'hands',
 'latest',
 'collection',
 'X',
 'Clothing',
 'Company',
 'designs',
 'smiling_face_with_hearteyes',
 'FashionForward']

# POS tagging

In [ ]:
tagged = nltk.pos_tag(tweets_data["Text"][0])
tagged

[('got', 'VBD'),
 ('hands', 'NNS'),
 ('latest', 'JJS'),
 ('collection', 'NN'),
 ('X', 'NN'),
 ('Clothing', 'NNP'),
 ('Company', 'NNP'),
 ('designs', 'VBZ'),
 ('smiling_face_with_hearteyes', 'NNS'),
 ('FashionForward', 'NNP')]

# Lemmatization

In [ ]:
def lemmatize(tokens,lemma):
  lemmatized = [lemma.lemmatize(token) for token in tokens]
  return lemmatized

In [ ]:
lemma = WordNetLemmatizer()
lemma.lemmatize("smart","a")

'smart'

In [ ]:
tweets_data["Text"][0]

['got',
 'hands',
 'latest',
 'collection',
 'X',
 'Clothing',
 'Company',
 'designs',
 'smiling_face_with_hearteyes',
 'FashionForward']

In [ ]:
temp = tweets_data["Text"].apply(lemmatize,args=([lemma]))
temp

0     [got, hand, latest, collection, X, Clothing, C...
1     [Disappointed, quality, fabric, new, X, Clothi...
2     [Can't, wait, strut, stuff, new, X, Clothing, ...
3     [customer, service, X, Clothing, Company, seri...
4     [Obsessed, vibrant, color, spring, collection,...
                            ...                        
95    [scored, killer, deal, workout, gear, X, Cloth...
96          [Disappointed, X, Clothing, Company, still]
97    [Obsessed, bohochic, vibe, new, X, Clothing, C...
98    [X, Clothing, Company, app, keep, crashing, ev...
99    [discovered, X, Clothing, Company, us, sustain...
Name: Text, Length: 100, dtype: object